In [ ]:
from google.colab import drive
import pandas as pd
from pandas import DataFrame
import pickle
import nltk


drive.mount('/content/drive/')
evaluation_data_path = '/content/drive/My Drive/기계학습/leaderboard_test_file.csv'
cls_weight_path = '/content/drive/My Drive/기계학습/LR.pkl'
cls_weight_path_2 = '/content/drive/My Drive/기계학습/DL.pkl'
features_path = '/content/drive/My Drive/기계학습/features.pkl'
result_path='/content/drive/My Drive/기계학습/result.csv'
nltk.download('punkt')
nltk.download('wordnet')


data = pd.read_csv(evaluation_data_path)
data.head()

lr = pickle.load(open(cls_weight_path, 'rb'))
features = pickle.load(open(features_path, 'rb'))
#DR = pickle.load(open(cls_weight_path_2, 'rb'))


def load_best_classifier():
    model = lr
    
    return model


def read_csv_and_classify(sparce_matrix, cls_model):
    print(sparce_matrix)
    print(len(sparce_matrix))
    print(len(sparce_matrix[0]))
    scores = cls_model.predict(sparce_matrix)
    print(scores)
    scores = scores.astype(np.uint8)
    print(scores)
    return scores

import os
import numpy as np
from google.colab import files


#with open(evaluation_data_path, "r") as data:
#     eval_lines = f.read().strip().split("\n")
#     eval_header = eval_lines[0]
#     eval_data = eval_lines[1:]
# eval_data = np.array([[ff  if ff!="" else "nan" for ff in fff.strip().split(",")] for fff in eval_data], dtype=np.float32)


import nltk as nlp
import re
description_list = []
for description in data['message']:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()   
    description = nlp.word_tokenize(description)
    #description = [ word for word in description if not word in set(stopwords.words("english"))]
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description)

from sklearn.feature_extraction.text import CountVectorizer 
max_features = 3000 #We use the most common word
print(features)
count_vectorizer = CountVectorizer( vocabulary = features ,max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print(sparce_matrix)
print(len(sparce_matrix))
print(len(sparce_matrix[0]))

print("the most using {} words: {}".format(max_features,count_vectorizer.get_feature_names()))
#sparce_matrix=[sparce_matrix zeros(size(sparse_matrix,1),3000-len(sparce_matrix[0]))]

# if len(sparce_matrix[0])<5995:
#     margin_matrix = np.zeros((len(sparce_matrix),5995-len(sparce_matrix[0])))
#     sparce_matrix = np.hstack((sparce_matrix,margin_matrix))
# print(sparce_matrix)
# print(len(sparce_matrix))
# print(len(sparce_matrix[0]))


best_classification_model = load_best_classifier()

cls_scores = read_csv_and_classify(sparce_matrix, best_classification_model)
#print(len(cls_scores))

cls_scores = cls_scores[0:len(data)]
assert cls_scores.shape==(len(data),), "Classification score shape mismatch"
assert cls_scores.dtype==np.uint8, "Classification score data type mismatch"

res_score = pd.DataFrame({'label' : cls_scores})

res_score.to_csv(result_path,index = False)
# data[:, 2] = cls_scores
# with open(evaluation_data_path, "w") as f:
#     f.write(eval_header+"\n"+"\n".join([",".join([str(ff) for ff in fff]) for fff in data]))
 
# files.download(evaluation_data_path)